In [3]:
import collections
import numpy as np
import pickle


In [8]:
experiments = ['BM25',
               'BERT_Base',
               ]

metrics = ['RaB', 'ARaB'] 
methods = ['tf', 'bool']


qry_bias_paths = {}
for metric in metrics:
    qry_bias_paths[metric] = {}
    for exp_name in experiments:
        qry_bias_paths[metric][exp_name] = {}
        for _method in methods:
            qry_bias_paths[metric][exp_name][_method] = 'data/msmarco_passage/run_bias_%s_%s_%s.pkl' % (exp_name, _method, metric)
        

queries_gender_annotated_path = "resources/queries_gender_annotated.csv"

at_ranklist = [5, 10, 20, 30, 40]

In [9]:
qry_bias_perqry = {}

for metric in metrics:
    qry_bias_perqry[metric] = {}
    for exp_name in experiments:
        qry_bias_perqry[metric][exp_name] = {}
        for _method in methods:
            _path = qry_bias_paths[metric][exp_name][_method]
            print (_path)
            with open(_path, 'rb') as fr:
                qry_bias_perqry[metric][exp_name][_method] = pickle.load(fr)



data/msmarco_passage/run_bias_BM25_tf_RaB.pkl
data/msmarco_passage/run_bias_BM25_bool_RaB.pkl
data/msmarco_passage/run_bias_BERT_Base_tf_RaB.pkl
data/msmarco_passage/run_bias_BERT_Base_bool_RaB.pkl
data/msmarco_passage/run_bias_BM25_tf_ARaB.pkl
data/msmarco_passage/run_bias_BM25_bool_ARaB.pkl
data/msmarco_passage/run_bias_BERT_Base_tf_ARaB.pkl
data/msmarco_passage/run_bias_BERT_Base_bool_ARaB.pkl


In [10]:
queries_effective = {}
with open(queries_gender_annotated_path, 'r') as fr:
    for line in fr:
        vals = line.strip().split(',')
        qryid = int(vals[0])
        qrytext = ' '.join(vals[1:-1])
        qrygender = vals[-1]
        if qrygender == 'n':
            queries_effective[qryid] = qrytext
len(queries_effective)

1765

In [11]:
eval_results_bias = {}
eval_results_feml = {}
eval_results_male = {}

for metric in metrics:
    eval_results_bias[metric] = {}
    eval_results_feml[metric] = {}
    eval_results_male[metric] = {}
    for exp_name in experiments:
        eval_results_bias[metric][exp_name] = {}
        eval_results_feml[metric][exp_name] = {}
        eval_results_male[metric][exp_name] = {}
        for _method in methods:
            eval_results_bias[metric][exp_name][_method] = {}
            eval_results_feml[metric][exp_name][_method] = {}
            eval_results_male[metric][exp_name][_method] = {}
            for at_rank in at_ranklist:
                _bias_list = []
                _feml_list = []
                _male_list = []

                for qryid in queries_effective.keys():
                    if qryid in qry_bias_perqry[metric][exp_name][_method][at_rank]:
                        _bias_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][0])
                        _feml_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][1])
                        _male_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][2])
                    else:
                        pass
                        #print ('missing', metric, exp_name, _method, at_rank, qryid)

                eval_results_bias[metric][exp_name][_method][at_rank] = np.mean([(_male_x-_feml_x) for _male_x, _feml_x in zip(_male_list, _feml_list)])
                eval_results_feml[metric][exp_name][_method][at_rank] = np.mean(_feml_list)
                eval_results_male[metric][exp_name][_method][at_rank] = np.mean(_male_list)




In [13]:

for metric in metrics:
    print (metric)
    for at_rank in at_ranklist:
        for _method in methods:
            for exp_name in experiments:
                print ("%25s\t%2d %5s\t%f\t%f\t%f" % (exp_name, at_rank, _method, eval_results_bias[metric][exp_name][_method][at_rank], eval_results_feml[metric][exp_name][_method][at_rank], eval_results_male[metric][exp_name][_method][at_rank]))
        print ("==========")

RaB
                     BM25	 5    tf	-0.021162	0.251156	0.229994
                BERT_Base	 5    tf	0.009629	0.167259	0.176888
                     BM25	 5  bool	-0.017674	0.205581	0.187907
                BERT_Base	 5  bool	0.010233	0.153488	0.163721
                     BM25	10    tf	-0.004641	0.219806	0.215165
                BERT_Base	10    tf	0.009101	0.165929	0.175030
                     BM25	10  bool	0.000930	0.182791	0.183721
                BERT_Base	10  bool	0.006202	0.156279	0.162481
                     BM25	20    tf	-0.000462	0.188988	0.188526
                BERT_Base	20    tf	0.015368	0.148183	0.163551
                     BM25	20  bool	0.004419	0.161628	0.166047
                BERT_Base	20  bool	0.013125	0.139383	0.152507
                     BM25	30    tf	0.002845	0.172844	0.175689
                BERT_Base	30    tf	0.013676	0.144351	0.158027
                     BM25	30  bool	0.006512	0.150388	0.156899
                BERT_Base	30  bool	0.013357	0.133569	0.146926
